<a href="https://colab.research.google.com/github/jhajagos/PHR2OHDSI/blob/main/Map_CDAs_to_OHDSI_CDM_in_a_Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Convert CDAs to OHDSI Compatible Parquet Files with PySpark

This notebook is for converting CDA XML to OHDSI. This pipeline has been tested with several CDAs (Epic, Oracle/Cerner) extracted from patient portals and Apple Health Kit.

More detail on how to map CDAs to OHDSI CDMs can be found at: https://github.com/jhajagos/PreparedSource2OHDSI/tree/main/map/prepared_source/cda

The conversion assumes that you are converting only a single person's CDAs to the OHDSI CDM.


In [ ]:
import json

In [ ]:
OHSDI_VOCABULARY_PATH = "/content/drive/MyDrive/OHDSI/vocabulary/20250317/export/"

CDA_FILE_PATH = "/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/"

SALT = "salty salt"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls {OHSDI_VOCABULARY_PATH}

concept_ancestor.parquet  concept_relationship.parquet	relationship.parquet
concept_class.parquet	  concept_synonym.parquet	vocabulary.parquet
concept_map.parquet	  domain.parquet
concept.parquet		  drug_strength.parquet


In [ ]:
%pip install pyspark==3.5.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.5-py2.py3-none-any.whl size=317747862 sha256=e923cfd8c6166159b9751b312f4f08b0e05a4e1ebab84b84f72f65d30c518290
  Stored in directory: /root/.cache/pip/wheels/0c/7f/b4/0e68c6d8d89d2e582e5498ad88616c16d7c19028680e9d3840
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1


In [ ]:
%pip install build

In [ ]:
%pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 13.9 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/jhajagos/PreparedSource2OHDSI.git

Cloning into 'PreparedSource2OHDSI'...
remote: Enumerating objects: 1470, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 1470 (delta 134), reused 198 (delta 81), pack-reused 1212 (from 1)
Receiving objects: 100% (1470/1470), 294.99 KiB | 17.35 MiB/s, done.
Resolving deltas: 100% (815/815), done.


In [ ]:
!cd ./PreparedSource2OHDSI && git pull

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 3), reused 5 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 442 bytes | 88.00 KiB/s, done.
From https://github.com/jhajagos/PreparedSource2OHDSI
   e6e206a..a7c9fec  main       -> origin/main
Updating e6e206a..a7c9fec
Fast-forward
 map/ohdsi/map_prepared_source_to_ohdsi_cdm.py | 3 ++-
 1 file changed, 2 insertions(+), 1 deletion(-)


In [ ]:
%%sh
cd ./PreparedSource2OHDSI
git pull
python -m build --wheel
cd ./dist/
pip install preparedsource2ohdsi-0.1.6-py3-none-any.whl

Already up to date.
* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools
  - wheel
* Getting build dependencies for wheel...
running egg_info
writing src/preparedsource2ohdsi.egg-info/PKG-INFO
writing dependency_links to src/preparedsource2ohdsi.egg-info/dependency_links.txt
writing requirements to src/preparedsource2ohdsi.egg-info/requires.txt
writing top-level names to src/preparedsource2ohdsi.egg-info/top_level.txt
reading manifest file 'src/preparedsource2ohdsi.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'src/preparedsource2ohdsi.egg-info/SOURCES.txt'
* Building wheel...
running bdist_wheel
running build
running build_py
copying src/preparedsource2ohdsi/ohdsi_cdm_5_3_1.py -> build/lib/preparedsource2ohdsi
copying src/preparedsource2ohdsi/__init__.py -> build/lib/preparedsource2ohdsi
copying src/preparedsource2ohdsi/prepared_source.py -> build/lib/preparedsource2ohdsi
copying src/preparedsource2ohdsi

In [ ]:
%%time
!python /content/PreparedSource2OHDSI/map/prepared_source/cda/cda_to_prepared_source_fragments.py -d {CDA_FILE_PATH} --salt "{SALT}"

Parsing: '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/mn_832f18645bcc13b27600207cd93606b7.xml'
Writing: '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps_frags/source_person.mn_832f18645bcc13b27600207cd93606b7.xml.csv'
Writing 1 rows in  '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps_frags/source_result.lab.mn_832f18645bcc13b27600207cd93606b7.xml.csv
Writing 1 rows in '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps_frags/source_medication.mn_832f18645bcc13b27600207cd93606b7.xml.csv'
Writing 1 rows in '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps_frags/source_condition.mn_832f18645bcc13b27600207cd93606b7.xml.csv'
Writing 1 rows in '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps_frags/source_procedure.mn_832f18645bcc13b27600207cd93606b7.xml.csv'
Writing 1 rows in  '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps_frags/source_result.vital.mn_832f18645bcc13b27600207cd93606b7.xml.

In [ ]:
%%time
!python /content/PreparedSource2OHDSI/map/prepared_source/cda/clean_combine_prepared_source_fragments.py -d {CDA_FILE_PATH}

Consolidating 'source_person.csv'
Reading '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps_frags/source_person.mn_832f18645bcc13b27600207cd93606b7.xml.csv'
Reading '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps_frags/source_person.mn_1562e983740c425fea1c4df43b6c7fe5.xml.csv'
Reading '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps_frags/source_person.mn_a65018ddb1c54ea51a45841441890077.xml.csv'
Reading '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps_frags/source_person.mn_71ca3d925c1b4c4b158ab330b89faa81.xml.csv'
Reading '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps_frags/source_person.mn_b0a6b93c261e53f053a128e070bdc217.xml.csv'
Reading '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps_frags/source_person.mn_a2628ac4f3a9fb0243d0d9dfbe225656.xml.csv'
Reading '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps_frags/source_person.mn_a186826f2db3a0e3a49ea66b448dd231.xml.csv'


In [ ]:
ps_configuration = {
  "concept_base_path": OHSDI_VOCABULARY_PATH,
  "export_concept_mapping_table_path": OHSDI_VOCABULARY_PATH,
  "concept_csv_file_extension": ".csv.bz2",
  "prepared_source_table_path": CDA_FILE_PATH + "output/prepared_source/",
  "prepared_source_csv_extension": ".csv",
  "staging_table_prefix": "",
  "ohdsi_output_location": CDA_FILE_PATH + "output/ohdsi/",
  "check_pointing": "NONE",
  "ohdsi_version": "5.4",
   "prepared_source_csv_table_path": CDA_FILE_PATH + "output/ps/",
  "prepared_source_tables_to_exclude": ["source_device","source_provider", "source_encounter_map",
  "source_person_map", "source_location", "source_care_site", "source_payer", "source_encounter", "source_encounter_detail"
  ],
  "jdbc": {
    "connection_string": "",
    "properties": {"username":  "", "password":  ""}
  }

}
with open("/content/ps_configuration.json", "w") as f:
  json.dump(ps_configuration, f)

In [ ]:
%%time
%%python /content/PreparedSource2OHDSI/map/prepared_source/csv/stage_streamlined_prepared_source.py -c /content/ps_configuration.json -l

Spark Configuration:
{'spark.driver.extraJavaOptions': '-Duser.timezone=GMT',
 'spark.driver.memory': '16g',
 'spark.executor.extraJavaOptions': '-Duser.timezone=GMT',
 'spark.sql.session.timeZone': 'UTC'}


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/01 11:02:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
INFO:root:Loading: '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps/source_person.csv'
INFO:root:Loading: '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps/source_observation_period.csv'
INFO:root:Loading: '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps/source_condition.csv'
INFO:root:Loading: '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps/source_procedure.csv'
INFO:root:Loading: '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps/source_medication.csv'
INFO:root:Loading: '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps/source_result.csv'
INFO:root:Loading: '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps/sou

CPU times: user 200 ms, sys: 28.3 ms, total: 228 ms
Wall time: 48.5 s


In [ ]:
%%time
%%python /content/PreparedSource2OHDSI/map/ohdsi/map_prepared_source_to_ohdsi_cdm.py -c /content/ps_configuration.json -l

Spark Configuration:
{'spark.driver.extraJavaOptions': '-Duser.timezone=GMT',
 'spark.driver.memory': '16g',
 'spark.executor.extraJavaOptions': '-Duser.timezone=GMT',
 'spark.sql.session.timeZone': 'UTC'}
Mapping Configuration:
{'check_pointing': 'NONE',
 'concept_base_path': '/content/drive/MyDrive/OHDSI/vocabulary/20250317/export/',
 'concept_csv_file_extension': '.csv.bz2',
 'export_concept_mapping_table_path': '/content/drive/MyDrive/OHDSI/vocabulary/20250317/export/',
 'jdbc': {'connection_string': '',
          'properties': {'password': '', 'username': ''}},
 'ohdsi_output_location': '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ohdsi/',
 'ohdsi_version': '5.4',
 'prepared_source_csv_extension': '.csv',
 'prepared_source_csv_table_path': '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ps/',
 'prepared_source_table_path': '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/prepared_source/',
 'prepared_source_tables_to_exclude': ['source_d

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/01 15:30:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
INFO:root:Check pointing mode: NONE
INFO:root:Loading 'concept' from '/content/drive/MyDrive/OHDSI/vocabulary/20250317/export/concept.parquet'
INFO:root:Loading 'concept_map' from '/content/drive/MyDrive/OHDSI/vocabulary/20250317/export/concept_map.parquet'
INFO:root:Loading 'vocabulary' from '/content/drive/MyDrive/OHDSI/vocabulary/20250317/export/vocabulary.parquet'
INFO:root:Total time to load concept and concept_map: 7.259278297424316 seconds
INFO:root:Loading 'source_care_site' from '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/prepared_source/source_care_site.parquet'
INFO:root:Loading 'source_condition' from '/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/prepared_source/source_con

CPU times: user 4.23 s, sys: 609 ms, total: 4.84 s
Wall time: 18min 51s


In [ ]:
!ls /content/

drive		      ps_configuration.json			    sample_data
PreparedSource2OHDSI  ps_configuration.json.generated.parquet.json


In [32]:
!cp ps_configuration.json.generated.parquet.json {CDA_FILE_PATH}/output/ohdsi/

In [ ]:
!du -h {CDA_FILE_PATH}output/ohdsi/

6.0K	/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ohdsi/support/visit_source_link.parquet
21K	/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ohdsi/support/source_condition_matched.parquet/mapped_domain_id=Condition
20K	/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ohdsi/support/source_condition_matched.parquet/mapped_domain_id=Measurement
20K	/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ohdsi/support/source_condition_matched.parquet/mapped_domain_id=Observation
20K	/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ohdsi/support/source_condition_matched.parquet/mapped_domain_id=Procedure
85K	/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ohdsi/support/source_condition_matched.parquet
17K	/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/ohdsi/support/source_procedure_matched.parquet/mapped_domain_id=__HIVE_DEFAULT_PARTITION__
18K	/content/drive/MyDrive/phr_ohdsi/source/jgh_documents/output/o

In [ ]:
%%python /content/PreparedSource2OHDSI/map/ohdsi/utilities/basic_mapped_data_stats.py -c /content/ps_configuration.json.generated.parquet.json -l